# Lindhardt susceptibility $\chi_0$ for a non interacting electron gas


HERE WE RECALL WAHT LINDHARD FUNCTION IS



The two-particle response of the non-interacting system is given by the Lindhardt susceptibility $\chi_0$. Diagrammatically it is a bubble diagram of two single-particle Green's function, that is most efficiently evaluated in imaginary time.

$$ \chi_0(\tau, r) \equiv 2 g_0(\tau, r) g_0(\beta - \tau, -r) $$

after evaluating the imaginary time bubble we transform $\chi_0$ back to momentum and frequency space, i.e.


$$ \chi_0(i\omega_n, r) \equiv \mathcal{F}_{\tau \rightarrow i\omega_n} \big\{ \chi_0(\tau, r) \big\}$$

$$ \chi_0(i\omega_n, k) \equiv \mathcal{F}_{r \rightarrow k} \big\{ \chi_0(i\omega_n, r) \big\}$$


While it is possible to do this computation in pure Python, it is relatively slow, due to the double loop in $r$ and $\tau$. 

Here we illustrate the C++ layer of the TRIQS library, using the TRIQS/cpp2py tool to wrap Python and C++
in a simple case.

The function bubble below computes the Lindhardt function from Eq (XX).
The following cell compiles the function "bubble" in C++, exposes ("wraps") it to Python and loads it
in the current scope.


In [1]:
%reload_ext cpp2py.magic

May take a few seconds to compile ...

In [3]:
%%cpp2py -C pytriqs
#include <triqs/gfs.hpp> 
using namespace triqs::gfs;

// The type of a Green function : (omega,k) -> Complex number
using g_tau_k_type = gf_view<cartesian_product<imfreq, brillouin_zone>>;

g_tau_k_type bubble(g_tau_k_type g) { 

    // Fourier Transformation of k, \omega to obtain g(t,r)
    auto gtr = make_gf_from_fourier<0,1>(g);
    
    // The mesh of gtr is a cartesian product mt x mr. We decompose it.
    auto [mt, mr] = gtr.mesh();
    
    // The inverse temperature from the mesh
    double beta = mt.domain().beta;
    
    // A new mesh for chi, with a bosonic statistics, but same size as mt.
    auto mtb = gf_mesh<imtime>{beta, Boson, mt.size()};
    
    // Build chi (tau, r) with this new mesh.
    auto chi0 = gf<cartesian_product<imtime, cyclic_lattice>>{{mtb, mr}};

    // we fill chi : chi(tau, r) = g(beta - tau, -r) * g(tau, r)
    for (auto const &r : mr)      
        for (auto const &t : mtb) 
            chi0[t, r] = gtr(beta - t, -r) * gtr(t, r); 

    // Fourier transform back to k, \omega space and return
    return make_gf_from_fourier<0,1>(chi0);
}

# Static Lindhardt susceptibility $\chi_0(\omega=0, k)$

The square lattice with only nearest-neighbour hopping $t$ has a property of "perfect nesting", meaning that large parts of the fermi surface are wrapped on-to each other by a single momentum transfer $k$. Go back to the fermi-surface polot of $\epsilon_k$ and determine this peculiar momentum vector.

The "perfect nesting" greatly enhances the particle-hole susceptibility of the system and the Static Lindhardt susceptibility $\chi_0(\omega=0, k)$ has a dominant peak at this momentum.

TODO : 

- Grab the Gf from HDF5 (to pass from a notebook to the next and keep the notebook simple !)

- Call Lindhard to obtain chi(q, omega)

- Plot chi(k, 0) with a slice
  and other plots.
  
- We save the function in HDF5 for later reuse.  
  
- Physics discussion ?

In [ ]:
chi0_tr, chi0_wr, chi0_wk = get_chi0(g0_tr, n_w, kmesh, rmesh)

In [ ]:
from pytriqs.plot.mpl_interface import oplot, oplotr, oploti, plt

chi0_w0k = chi0_wk[Idx(0), :][0, 0]
data, k_vec, (kx, ky, kz) = extend_data_on_boundary(chi0_w0k.data, n_k)
extent_k = np.array([kx.min(), kx.max(), ky.min(), ky.max()])

plt.figure(figsize=(3.25*4, 5*2))
plt.title('Static Lindhardt susceptibility $\chi_0(\omega=0, k)$')
plt.imshow(data[:, :, 0].real, cmap=plt.get_cmap('terrain_r'), origin='lower', 
           vmin=0, extent=extent_k)
plt.colorbar(); plt.tight_layout()
plt.savefig('figure_chi0_w0k.pdf')
plt.xlabel(r'$k_x/(2\pi)$')
plt.ylabel(r'$k_y/(2\pi)$')

In [ ]:
from k_space_viz import get_rel_k_interpolator

interp = get_rel_k_interpolator(
    chi0_wk[Idx(0), :][0, 0].data, kmesh, H_0.bz, n_k,
    extend_boundary=True, interpolator='linear2D')
chi0_plot = interp(k_path_vecs[:, :2])

from pytriqs.plot.mpl_interface import plt

plt.figure(figsize=(3.25*3, 8))
plt.title('Static Lindhardt susceptibility $\chi_0(\omega=0, k)$')
plt.plot(k_plot, chi0_plot.real)

plt.grid(); plt.axes().set_xticks(K_plot)
plt.xlim([K_plot.min(), K_plot.max()])
plt.axes().set_xticklabels([r'$\Gamma$',r'$X$',r'$M$',r'$\Gamma$'])
plt.ylabel(r'$\epsilon_k$'); plt.tight_layout()
plt.savefig('figure_chi0_k_bandpath.pdf')